# Optim and Enzyme with ECCO.jl

Here we check that we can retrieve and evaluate functions and their adjoints from the ECCO toy problems.

In [16]:
using Pkg; Pkg.add(url="https://github.com/eldavenport/ECCO.jl")
using ECCO

    Updating git-repo `https://github.com/eldavenport/ECCO.jl`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
# toy problem to evaluate f() at (x,y) and the adjoint of f at (x,y)
(f,f_ad,x,y)=ECCO.toy_problems.enzyme_ex4()

fc=f(x,y)
adx=f_ad(x,y)
(fc=fc,adx=adx)

(fc = -3.0606099804357885, adx = [0.0, 917.7850566361462, -6.121219960871577, -0.44841281435892])

In [3]:
# toy problem that uses optim with the analytical gradient (g!)
(f,g!,x0,x1,result)=ECCO.toy_problems.optim_ex2()
dx=1e-4*(x0-x1)
(fc=f(x1),gradient_check=f(x1)<f(x1+dx))

(fc = 5.191703158437428e-27, gradient_check = true)

In [4]:
# toy problem that uses optim with enzyme gradient (h!)
(h,h!,x0,x1,result) = ECCO.toy_problems.optim_ex3()
(x0=x0,x1=x1)

(x0 = [0.0, 0.0], x1 = [0.999999999999928, 0.9999999999998559])

# Optim with Enzyme and AirSeaFlux

Setting up a function (f_tau) that utilizes AirSeaFlux, getting its adjoint with Enzyme, and then doing a basic optimization with a fake observation.

In [2]:
Pkg.add("AirSeaFluxes")
import AirSeaFluxes: bulkformulae
using Enzyme, Optim

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [6]:
#= Documentation from AirSeaFluxes.jl 
bulkformulae(atemp,aqh,speed,sst,hu=10,ht=2,hq=2,zref=10,atmrho=1.2)

Units:
atemp  - mean air temperature (K)  at height ht (m)
aqh    - mean air humidity (kg/kg) at height hq (m)
speed  - mean wind speed (m/s)     at height hu (m)
sst    - sea surface temperature (K)

Bulk formulae formulation:
```
wind stress = (ust,vst) = rhoA * Cd * Ws * (del.u,del.v)
Sensib Heat flux = fsha = rhoA * Ch * Ws * del.T * CpAir
Latent Heat flux = flha = rhoA * Ce * Ws * del.Q * Lvap
                 = -Evap * Lvap
```

with Cd,Ch,Ce = drag coefficient, Stanton number and
Dalton number respectively [no-units], function of
height & stability; and

```
Ws = wind speed = sqrt(del.u^2 +del.v^2)
del.T = Tair - Tsurf ; del.Q = Qair - Qsurf
```
=#

function f_tau(x::Array{Float64})
    y = bulkformulae(x[1],x[2],x[3],x[4]).tau
end
function f_tau_ad!(bx2, x) 
    bx = zeros(size(x))
    Enzyme.autodiff(Reverse, f_tau, Duplicated(x, bx))
    bx2 .= bx
end
x0 = [300.,0.001,1.,10.]
bx2 = zeros(size(x0))
f_tau_ad!(bx2,x0)

result=Optim.optimize(f_tau,f_tau_ad!,x0)
x1=Optim.minimizer(result)

4-element Vector{Float64}:
 300.0
   0.001
  -0.14443527308573367
  10.0

Now we can add an observation to the above problem to make it marginally more meaningful.

In [10]:
# Create a cost function that is the squared error between an observation and tau
function J_tau(x::Vector{Float64})
    # hard coded observation
    y_obs = 0.1
    y = bulkformulae(x[1],x[2],x[3],x[4]).tau
    
    # cost function
    J = abs(y-y_obs)^2
end

# get the adjoint of the cost function
function J_tau_ad!(bx2, x) 
    bx = zeros(size(x))
    Enzyme.autodiff(Reverse, J_tau, Duplicated(x, bx))
    bx2 .= bx
end

x0 = [300.,0.001,1.,10.]

# evaluate the gradient at x0 (just for testing)
bx2 = zeros(size(x0))
J_tau_ad!(bx2,x0)

# optimization with the cost function and the enzyme generated gradient 
result=Optim.optimize(J_tau, J_tau_ad!, x0, Optim.Options(show_trace=true))
x1=Optim.minimizer(result)

# check that tau at x1 is close to y_obs
y1 = bulkformulae(x1[1],x1[2],x1[3],x1[4]).tau

Iter     Function value   Gradient norm 
     0     8.252726e-03     1.663451e-03
 * time: 4.887580871582031e-5
     1     5.474962e-03     1.195887e-03
 * time: 0.0002779960632324219
     2     1.856151e-06     2.010559e-05
 * time: 0.0003829002380371094
     3     2.534444e-12     2.346009e-08
 * time: 0.00048804283142089844
     4     8.324333e-28     4.251711e-16
 * time: 0.0005888938903808594


0.09999999999997115

The section takes the above code and makes it more configurable by making the observation an argument to the cost function. We then use a closure to get a cost function that is fixed to the given observation. We take the adjoint of the closure with Enzyme, and then do an optimization to get the inputs that minimize the error between bulkformulae.tau and the observation. The syntax here is important! Enzyme is picky about how the closure is defined, which is apparently a known issue. 

In [19]:
# cost function with the obs as an argument
x0 = [300.,0.001,1.,10.]

function J(x::Vector{Float64},y_obs)
    y = bulkformulae(x[1],x[2],x[3],x[4]).tau
    # cost function
    J = (y-y_obs)^2
    return J
end

# create a new "closure" that will keep y_obs constant
y_obs = 0.1

function cost_closure(y_obs)
    return x -> J(x,y_obs)
end
cost = cost_closure(y_obs)

# evaluate cost at x0 to see if this works
# cost(x0) = 0.008252726417096927

# get the adjoint of this new closure function that accounts for the constant obs
function cost_ad!(bx2, x) 
    bx = zeros(size(x))
    Enzyme.autodiff(Reverse, cost, Duplicated(x, bx))
    bx2 .= bx
end

# for testing: evaluate the gradient at x0
bx2 = zeros(size(x0))
cost_ad!(bx2,x0)

# optimization with the cost function and it's adjoint 
# observations are accounted for and constant
result=Optim.optimize(cost, cost_ad!, x0)
x1=Optim.minimizer(result)

# check that tau at x1 is close to y_obs
y1 = bulkformulae(x1[1],x1[2],x1[3],x1[4]).tau

0.09999999999997115